# Code to reproduce the figures found by Myles in Excel
Read in CO2 emissions per year, define a series of constants (not sure where these come from at the moment) and reproduce the graphs that Myles made in excel. The calculation of non-CO2 ERF changes prior to 2000, adding some complication to the code.
Next steps are to add error bars.


In [239]:
import numpy as np
import pandas as pd

In [240]:
### define constants to be used. These are found in cells ' Time Series' BB6-BB9 in Myles spreadsheet, 

alpha = 0   
AGWP100 = 91.7   
H = 100  
TCRE = 0.00045  

### More constants
contrail_ERF = 0.000000000936 ### contrail ERF, see Master ERF Table C7
nonco2_erf_yr2000 = 35.00 ### 'Time Series' AZ30
co2_emm_yr2000 = 685.5 ### 'Time Series' AX30
contrail_erf_yr2000 = 28.35 ### 'time series' AO30
initial_non_co2_erf = 14.15072653 ### 'time series' AZ10
initial_date = 1980

In [241]:
### read in the data (I'm not sure how to do this all from GITHUB, so code currently reads it in from my computer's harddrive)

co2_forcing = pd.read_csv(r'C:\Users\galla\Documents\University Physics\Fourth Year Notes\MPhys_Project\2020_12_01InitialPaper\FlyingClimate-main\FlyingClimate-main\data\CO2.csv')

co2_emm = pd.read_csv(r'C:\Users\galla\Documents\University Physics\Fourth Year Notes\MPhys_Project\2020_12_01InitialPaper\CO2_emm_data.csv')
co2_forcing = co2_forcing[1:].astype(float)
#co2_forcing
#co2_emm

In [242]:
### remove null lines

co2 = co2_emm.dropna(subset=['Tg/yr'])
#co2

### Pre 2000s
We will generate columns with the following headings for the dates pre 2000

'CO2 emm', 'non CO2 ERF', 'Cum CO2', 'Cum CO2 -fe', 'CO2 Warming', 'Non-CO2 warming'

In [243]:
### take only pre 2000 years of co2 data
co2_pre2000 = co2.iloc[:20].reset_index(drop=True)

In [244]:
### non co2 ERf
# first find contrail erf
contrail_erf = co2_pre2000['Contrail Distance/ km'] * contrail_ERF
## next add contrail erf and other non co2 erf's
non_co2_erf = contrail_erf + ((nonco2_erf_yr2000 - contrail_erf_yr2000) * co2_pre2000['Tg/yr'] / co2_emm_yr2000)
#non_co2_erf_df = pd.DataFrame(non_co2_erf, columns=['non CO2 ERF'])
co2_pre2000['non CO2 ERF'] = non_co2_erf
#co2_pre2000

In [245]:
### Cumulative CO2

cum_co2 = []
for i in range(1, len(co2_pre2000) + 1) :
    co2_sum = co2_pre2000.iloc[:i, 1]
    cum_co2_i = co2_sum.sum(axis=0)/1000
    cum_co2.append(cum_co2_i)

        
co2_pre2000['Cum CO2'] = cum_co2
#co2_pre2000
    

In [246]:
# Cumulative CO2-fe  - not sure what this is
cum_fe = (1 - alpha) * (H / AGWP100) * (co2_pre2000['non CO2 ERF'] - initial_non_co2_erf) + (alpha/AGWP100) * \
            initial_non_co2_erf * (co2_pre2000['Date'] - initial_date)
co2_pre2000['Cum CO2-fe'] = cum_fe
#co2_pre2000

In [247]:
### CO2 Warming
co2_pre2000['CO2 Warming'] = co2_pre2000['Cum CO2'] * TCRE
#co2_pre2000

In [248]:
### non CO2 Warming
co2_pre2000['non-CO2 Warming'] = co2_pre2000['Cum CO2-fe'] * TCRE
co2_pre2000

,Date,Tg/yr,Contrail Distance/ km,median total non co2 erf,non CO2 ERF,Cum CO2,Cum CO2-fe,CO2 Warming,non-CO2 Warming
0,1980,406.8,1.090000e+10,NaN,14.148746,0.4068,-0.002160,0.000183,-9.720378e-07
1,1981,399.9,1.060000e+10,NaN,13.801009,0.8067,-0.381371,0.000363,-1.716170e-04
2,1982,405.1,1.070000e+10,NaN,13.945054,1.2118,-0.224288,0.000545,-1.009298e-04
3,1983,411.9,1.100000e+10,NaN,14.291821,1.6237,0.153865,0.000731,6.923917e-05
4,1984,439.4,1.180000e+10,NaN,15.307397,2.0631,1.261363,0.000928,5.676135e-04
5,1985,453.3,1.240000e+10,NaN,16.003840,2.5164,2.020843,0.001132,9.093795e-04
6,1986,478.7,1.340000e+10,NaN,17.186244,2.9951,3.310270,0.001348,1.489621e-03
7,1987,500.7,1.430000e+10,NaN,18.242065,3.4958,4.461656,0.001573,2.007745e-03
8,1988,525.0,1.520000e+10,NaN,19.320198,4.0208,5.637373,0.001809,2.536818e-03
9,1989,546.7,1.570000e+10,NaN,19.998708,4.5675,6.377298,0.002055,2.869784e-03


# Post 2000 


In [249]:
### post 2000 co2 data

co2_post2000 = co2.iloc[20:].reset_index(drop=True)
#co2_post2000

In [250]:
### non co2_erf
co2_forcing['mean'] = (co2_forcing['LinClim.1'] + co2_forcing['CICERO.1'] + co2_forcing['FAIR.1']) / 3
AF = co2_forcing['mean'][60:].reset_index(drop=True).astype(float)
AU = co2_post2000['median total non co2 erf'].reset_index(drop=True).astype(float)
co2_post2000['non CO2 ERF'] = AU - AF
#co2_post2000

In [259]:
### cumulative co2
cum_co2_all = np.cumsum(co2['Tg/yr'])/1000
cum_co2_post = cum_co2_all[20:]
co2_post2000['Cum CO2'] = cum_co2_post.reset_index(drop=True)
#cum_co2_post.reset_index(drop=True)

In [260]:
### cum non co2
cum_fe_post = (1 - alpha) * (H / AGWP100) * (co2_post2000['non CO2 ERF'] - initial_non_co2_erf) + (alpha/AGWP100) * \
            initial_non_co2_erf * (co2_post2000['Date'] - initial_date)
co2_post2000['Cum CO2-fe'] = cum_fe_post
#co2_post2000

In [261]:
### co2 warming
co2_post2000['CO2 Warming'] = co2_post2000['Cum CO2'] * TCRE
#co2_post2000

In [265]:
# non co2 warming

co2_post2000['non-CO2 Warming'] = co2_post2000['Cum CO2-fe'] * TCRE
#co2_post2000

In [266]:
co2_pre2000.append(co2_post2000).reset_index(drop=True)

,Date,Tg/yr,Contrail Distance/ km,median total non co2 erf,non CO2 ERF,Cum CO2,Cum CO2-fe,CO2 Warming,non-CO2 Warming
0,1980,406.8,1.090000e+10,NaN,14.148746,0.4068,-0.002160,0.000183,-9.720378e-07
1,1981,399.9,1.060000e+10,NaN,13.801009,0.8067,-0.381371,0.000363,-1.716170e-04
2,1982,405.1,1.070000e+10,NaN,13.945054,1.2118,-0.224288,0.000545,-1.009298e-04
3,1983,411.9,1.100000e+10,NaN,14.291821,1.6237,0.153865,0.000731,6.923917e-05
4,1984,439.4,1.180000e+10,NaN,15.307397,2.0631,1.261363,0.000928,5.676135e-04
5,1985,453.3,1.240000e+10,NaN,16.003840,2.5164,2.020843,0.001132,9.093795e-04
6,1986,478.7,1.340000e+10,NaN,17.186244,2.9951,3.310270,0.001348,1.489621e-03
7,1987,500.7,1.430000e+10,NaN,18.242065,3.4958,4.461656,0.001573,2.007745e-03
8,1988,525.0,1.520000e+10,NaN,19.320198,4.0208,5.637373,0.001809,2.536818e-03
9,1989,546.7,1.570000e+10,NaN,19.998708,4.5675,6.377298,0.002055,2.869784e-03
